# 文本分类实例

## Step1 导入相关包

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import load_dataset

## Step2 加载数据集

In [2]:
dataset = load_dataset("csv", data_files="./ChnSentiCorp_htl_all.csv", split="train")
dataset = dataset.filter(lambda x: x["review"] is not None)
dataset

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to C:/Users/yuyao/.cache/huggingface/datasets/csv/default-4d32ce53bf893ba4/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


Filter:   0%|          | 0/7766 [00:00<?, ? examples/s]

Dataset({
    features: ['label', 'review'],
    num_rows: 7765
})

## Step3 划分数据集

In [3]:
datasets = dataset.train_test_split(test_size=0.1)
datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'review'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['label', 'review'],
        num_rows: 777
    })
})

## Step4 数据集预处理

In [4]:
import torch

tokenizer = AutoTokenizer.from_pretrained("hfl/rbt3")

def process_function(examples):
    tokenized_examples = tokenizer(examples["review"], max_length=128, truncation=True)
    tokenized_examples["labels"] = examples["label"]
    return tokenized_examples

tokenized_datasets = datasets.map(process_function, batched=True, remove_columns=datasets["train"].column_names)
tokenized_datasets

Map:   0%|          | 0/6988 [00:00<?, ? examples/s]

Map:   0%|          | 0/777 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 6988
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 777
    })
})

## Step5 创建模型

In [40]:
model = AutoModelForSequenceClassification.from_pretrained("hfl/rbt3")

Some weights of the model checkpoint at hfl/rbt3 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at hfl/rbt3

## Step6 创建评估函数

In [41]:
import evaluate

acc_metric = evaluate.load("accuracy")
f1_metirc = evaluate.load("f1")

In [42]:
def eval_metric(eval_predict):
    predictions, labels = eval_predict
    predictions = predictions.argmax(axis=-1)
    acc = acc_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metirc.compute(predictions=predictions, references=labels)
    acc.update(f1)
    return acc

## Step7 创建TrainingArguments

In [43]:
train_args = TrainingArguments(output_dir="./checkpoints",      # 输出文件夹
                               per_device_train_batch_size=64,  # 训练时的batch_size
                               per_device_eval_batch_size=128,  # 验证时的batch_size
                               logging_steps=10,                # log 打印的频率
                               evaluation_strategy="epoch",     # 评估策略
                               save_strategy="epoch",           # 保存策略
                               save_total_limit=3,              # 最大保存数
                               learning_rate=2e-5,              # 学习率
                               weight_decay=0.01,               # weight_decay
                               metric_for_best_model="f1",      # 设定评估指标
                               load_best_model_at_end=True)     # 训练完成后加载最优模型
train_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_config={'fsdp_min_num_params': 0, 'xla': False, 'xla_fsdp_grad_ckpt': False},
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=True,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ign

## Step8 创建Trainer

In [44]:
from transformers import DataCollatorWithPadding
trainer = Trainer(model=model, 
                  args=train_args, 
                  train_dataset=tokenized_datasets["train"], 
                  eval_dataset=tokenized_datasets["test"], 
                  data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
                  compute_metrics=eval_metric)

## Step9 模型训练

In [45]:
trainer.train()

c:\Users\yuyao\miniconda3\envs\transformers\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


  0%|          | 0/330 [00:00<?, ?it/s]

{'loss': 0.6524, 'learning_rate': 1.9393939393939395e-05, 'epoch': 0.09}
{'loss': 0.5357, 'learning_rate': 1.8787878787878792e-05, 'epoch': 0.18}
{'loss': 0.4865, 'learning_rate': 1.8181818181818182e-05, 'epoch': 0.27}
{'loss': 0.3981, 'learning_rate': 1.7575757575757576e-05, 'epoch': 0.36}
{'loss': 0.4093, 'learning_rate': 1.6969696969696972e-05, 'epoch': 0.45}
{'loss': 0.3473, 'learning_rate': 1.6363636363636366e-05, 'epoch': 0.55}
{'loss': 0.329, 'learning_rate': 1.575757575757576e-05, 'epoch': 0.64}
{'loss': 0.3606, 'learning_rate': 1.5151515151515153e-05, 'epoch': 0.73}
{'loss': 0.3507, 'learning_rate': 1.4545454545454546e-05, 'epoch': 0.82}
{'loss': 0.2825, 'learning_rate': 1.3939393939393942e-05, 'epoch': 0.91}
{'loss': 0.2716, 'learning_rate': 1.3333333333333333e-05, 'epoch': 1.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.28014910221099854, 'eval_accuracy': 0.8815958815958816, 'eval_f1': 0.9157509157509157, 'eval_runtime': 0.2829, 'eval_samples_per_second': 2746.925, 'eval_steps_per_second': 24.747, 'epoch': 1.0}
{'loss': 0.2508, 'learning_rate': 1.2727272727272728e-05, 'epoch': 1.09}
{'loss': 0.2587, 'learning_rate': 1.2121212121212122e-05, 'epoch': 1.18}
{'loss': 0.2819, 'learning_rate': 1.1515151515151517e-05, 'epoch': 1.27}
{'loss': 0.2948, 'learning_rate': 1.0909090909090909e-05, 'epoch': 1.36}
{'loss': 0.2465, 'learning_rate': 1.0303030303030304e-05, 'epoch': 1.45}
{'loss': 0.2512, 'learning_rate': 9.696969696969698e-06, 'epoch': 1.55}
{'loss': 0.2704, 'learning_rate': 9.090909090909091e-06, 'epoch': 1.64}
{'loss': 0.3138, 'learning_rate': 8.484848484848486e-06, 'epoch': 1.73}
{'loss': 0.2473, 'learning_rate': 7.87878787878788e-06, 'epoch': 1.82}
{'loss': 0.279, 'learning_rate': 7.272727272727273e-06, 'epoch': 1.91}
{'loss': 0.2283, 'learning_rate': 6.666666666666667e-06, 'epoch': 

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2614673376083374, 'eval_accuracy': 0.8957528957528957, 'eval_f1': 0.926961226330027, 'eval_runtime': 0.2775, 'eval_samples_per_second': 2800.155, 'eval_steps_per_second': 25.227, 'epoch': 2.0}
{'loss': 0.2582, 'learning_rate': 6.060606060606061e-06, 'epoch': 2.09}
{'loss': 0.2319, 'learning_rate': 5.4545454545454545e-06, 'epoch': 2.18}
{'loss': 0.2539, 'learning_rate': 4.848484848484849e-06, 'epoch': 2.27}
{'loss': 0.226, 'learning_rate': 4.242424242424243e-06, 'epoch': 2.36}
{'loss': 0.256, 'learning_rate': 3.6363636363636366e-06, 'epoch': 2.45}
{'loss': 0.2488, 'learning_rate': 3.0303030303030305e-06, 'epoch': 2.55}
{'loss': 0.2414, 'learning_rate': 2.4242424242424244e-06, 'epoch': 2.64}
{'loss': 0.1965, 'learning_rate': 1.8181818181818183e-06, 'epoch': 2.73}
{'loss': 0.1908, 'learning_rate': 1.2121212121212122e-06, 'epoch': 2.82}
{'loss': 0.2204, 'learning_rate': 6.060606060606061e-07, 'epoch': 2.91}
{'loss': 0.203, 'learning_rate': 0.0, 'epoch': 3.0}


  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2530556321144104, 'eval_accuracy': 0.8931788931788932, 'eval_f1': 0.9243391066545124, 'eval_runtime': 0.2816, 'eval_samples_per_second': 2759.009, 'eval_steps_per_second': 24.856, 'epoch': 3.0}
{'train_runtime': 19.8469, 'train_samples_per_second': 1056.283, 'train_steps_per_second': 16.627, 'train_loss': 0.2991888494202585, 'epoch': 3.0}


TrainOutput(global_step=330, training_loss=0.2991888494202585, metrics={'train_runtime': 19.8469, 'train_samples_per_second': 1056.283, 'train_steps_per_second': 16.627, 'train_loss': 0.2991888494202585, 'epoch': 3.0})

## Step10 模型评估

In [46]:
trainer.evaluate(tokenized_datasets["test"])

  0%|          | 0/7 [00:00<?, ?it/s]

{'eval_loss': 0.2614673376083374,
 'eval_accuracy': 0.8957528957528957,
 'eval_f1': 0.926961226330027,
 'eval_runtime': 0.52,
 'eval_samples_per_second': 1494.175,
 'eval_steps_per_second': 13.461,
 'epoch': 3.0}

## Step11 模型预测

In [16]:
trainer.predict(tokenized_datasets["test"])

  0%|          | 0/98 [00:00<?, ?it/s]

PredictionOutput(predictions=array([[-3.0055466,  1.8600152],
       [-3.9204335,  2.8778634],
       [-3.8034165,  2.7620633],
       ...,
       [-3.879293 ,  3.0156114],
       [ 3.2226527, -2.82157  ],
       [-4.322149 ,  3.4938972]], dtype=float32), label_ids=array([0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1,
       1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0,
       0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0,
       1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1,
       0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
    

## Step6 训练与验证

In [9]:
def evaluate():
    model.eval()
    with torch.inference_mode():
        for batch in validloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            output = model(**batch)
            pred = torch.argmax(output.logits, dim=-1)
            clf_metrics.add_batch(predictions=pred.long(), references=batch["labels"].long())
    return clf_metrics.compute()

def train(epoch=3, log_step=100):
    global_step = 0
    for ep in range(epoch):
        model.train()
        for batch in trainloader:
            if torch.cuda.is_available():
                batch = {k: v.cuda() for k, v in batch.items()}
            optimizer.zero_grad()
            output = model(**batch)
            output.loss.backward()
            optimizer.step()
            if global_step % log_step == 0:
                print(f"ep: {ep}, global_step: {global_step}, loss: {output.loss.item()}")
            global_step += 1
        clf = evaluate()
        print(f"ep: {ep}, {clf}")

## Step7 模型训练

In [10]:
train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


ep: 0, global_step: 0, loss: 0.4559326171875
ep: 0, global_step: 100, loss: 0.20979377627372742
ep: 0, global_step: 200, loss: 0.14855852723121643
ep: 0, {'accuracy': 0.8918918918918919, 'f1': 0.920754716981132}
ep: 1, global_step: 300, loss: 0.2373150736093521
ep: 1, global_step: 400, loss: 0.19246485829353333
ep: 1, {'accuracy': 0.9047619047619048, 'f1': 0.9270216962524656}
ep: 2, global_step: 500, loss: 0.4106125235557556
ep: 2, global_step: 600, loss: 0.23521238565444946
ep: 2, {'accuracy': 0.9137709137709138, 'f1': 0.9339901477832513}


## Step8 模型预测

In [11]:
sen = "我觉得这家酒店不错，饭很好吃！"
id2_label = {0: "差评！", 1: "好评！"}
model.eval()
with torch.inference_mode():
    inputs = tokenizer(sen, return_tensors="pt")
    inputs = {k: v.cuda() for k, v in inputs.items()}
    logits = model(**inputs).logits
    pred = torch.argmax(logits, dim=-1)
    print(f"输入：{sen}\n模型预测结果:{id2_label.get(pred.item())}")

输入：我觉得这家酒店不错，饭很好吃！
模型预测结果:好评！


In [12]:
from transformers import pipeline

model.config.id2label = id2_label
pipe = pipeline("text-classification", model=model, tokenizer=tokenizer, device=0)

In [13]:
pipe(sen)

[{'label': '好评！', 'score': 0.9770323038101196}]